## Word2Vec Example in Python

[http://districtdatalabs.silvrback.com/modern-methods-for-sentiment-analysis]

In [6]:
import gensim
from gensim.models.word2vec import Word2Vec

In [13]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('../../GoogleNews-vectors-negative300.bin', binary=True) 

In [49]:
model.most_similar('ok',topn=5)

[('okay', 0.8567795157432556),
 ('alright', 0.8077974915504456),
 ('OK', 0.6864468455314636),
 ('lol', 0.6789621114730835),
 ('anyways', 0.6699042916297913)]

In [40]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)

[('queen', 0.7118192315101624),
 ('monarch', 0.6189672946929932),
 ('princess', 0.5902429819107056),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377322435379028)]

In [ ]:
file_path = '../../twit_data/training.csv'
data = pd.read_csv(file_path,encoding='latin1',header=None)
columns = ['sentiment','text']
data=data[[0,5]]
data.columns=columns

In [ ]:
## keep only positive and negeative values 
data = data[((data['sentiment']==4) | (data['sentiment']==0))]
data['sentiment'][data['sentiment']==4]  = 1 
data

### To get vector from google word to vect 

In [58]:
## word to vector
model['me'].shape

(300,)

In [ ]:
## document to matrix, by looking up wordvectors



In [15]:
# Retrieve the entire list of "words" from the Google Word2Vec model, and write
# these out to text files so we can peruse them.
vocab = model.vocab.keys()
wordsInVocab = len(vocab)

In [29]:
model.index2word[0]

'</s>'

In [56]:
list(model.vocab)[0]

'</s>'